In [1]:
import numpy as np
from itertools import permutations

In [2]:
def reset_memory(day):
    '''Load program from file'''
    program = np.genfromtxt('day'+str(day)+'_input.txt', dtype=np.int, delimiter=',')
    if day == 2:
        #Reset the 1202 error
        program[1] = 12
        program[2] = 2
    return program

In [3]:
def run_program(program, param_in=None, i=None, pause_on_out=False, verbose=True):
    '''Function to run the program
       Steps along until 99 is reached
       fctn: 1 == +, 2 == *, 3 == input, 4 == output,
             5 == jump-if-true, 6 == jump-if-false,
             7 == <, 8 == ==,
             99 == STOP
       mode: 0 == position, 1 == immediate
       ABCDE: DE - fctn (with leading 0 if < 10)
              C  - mode of param1
              B  - mode of param2
              A  - mode of param3'''
    if i is None:
        i = 0
    state = 0
    param_out = []
    while i < len(program):
        fctn = program[i]
        #ABCDE
        
        #Break function down from ABCDE into A B C DE
        instruction = np.zeros(5, dtype=np.int)
        for j in range(4, -1, -1):
            instruction[4-j] = fctn // pow(10, j)
            fctn = int(fctn % pow(10, j))
        fctn = 10*instruction[3] + instruction[4] #DE

        #fctn that perform operations or comparisons
        if fctn == 1 or fctn == 2 or (fctn >= 5 and fctn <= 8):
            #If parameter 1 is in place...
            if instruction[2] == 1: #C
                param1 = program[i+1]
            #...or from memory
            else:
                param1 = program[program[i+1]]

            #If parameter 2 is in place...
            if instruction[1] == 1: #B
                param2 = program[i+2]
            #...or from memory
            else:
                param2 = program[program[i+2]]
                
            #If parameter 3 in in place...
            if instruction[0] == 1: #A
                    param3 = i+3
            #...or from memory
            else:
                param3 = program[i+3]

            #fctn that output to param3
            if fctn == 1 or fctn == 2 or fctn == 7 or fctn == 8:
                #Sum
                if fctn == 1:
                    result = param1+param2
                    
                #Multiply
                elif fctn == 2:
                    result = param1*param2
                    
                #less-than
                elif fctn == 7:
                    if param1 < param2:
                        result = 1
                    else:
                        result = 0
                        
                #equal-to
                elif fctn == 8:
                    if param1 == param2:
                        result = 1
                    else:
                        result = 0
                    
                program[param3] = result                
                i += 4
                
            #fctn that jump
            elif fctn == 5 or fctn == 6:
                #   jump-if-true                   jump-if-false
                if (fctn == 5 and param1 != 0) or (fctn == 6 and param1 == 0):
                    i = param2
                else:
                    i += 3
               
        #I/O functions
        #Get input
        elif fctn == 3:
            #print('Input:')
            if param_in is None:
                print('Input:')
                param0 = input()
            elif type(param_in) == np.ndarray or type(param_in) == list:
                param0 = param_in[state]
                #print(param0)
                state += 1
                if state >= len(param_in):
                    state = 0
            else:
                param0 = param_in
                #print(param0)
            
            #Save input in place...
            if instruction[2] == 1:
                program[i+1] = param0
            #...or to memory
            else:
                program[program[i+1]] = param0
            i += 2
            
        #Print output
        elif fctn == 4:
            #Output in place...
            if instruction[2] == 1:
                paramN = program[i+1]
            #...or from memory
            else:
                paramN = program[program[i+1]]
            
            param_out.append(paramN)
            i += 2
            
            #Print output
            if verbose:
                print('Output:', paramN)
            
            #If  using output as input for another program, this program can pause            
            if pause_on_out:
                return program, np.array(param_out), i
            
        #End program
        elif fctn == 99:
            i = -1
            break
            
        #Error handling
        else:
            print('ERROR:')
            print('i:', i, 'fctn:', fctn, 'instruction:', instruction)
            break
        
    return program, np.array(param_out), i

In [4]:
phases = permutations([0,1,2,3,4], 5)

max_out = 0
for phase in phases:
    in_signal = 0
    for i in range(0, len(phase)):
        program = reset_memory(7)
        _, out_signal, _ = run_program(program, [phase[i], in_signal], verbose=False)
        in_signal = out_signal[0]
    if in_signal > max_out:
        max_out = in_signal

print('Part 1 Solution:', max_out)

Part 1 Solution: 99376


In [5]:
phases = permutations([5,6,7,8,9], 5)
max_out  = 0

for phase in phases:
    program = reset_memory(7)
    program = np.vstack((program, program, program, program, program))
    pointer = np.array([0,0,0,0,0])

    in_signal = 0
    #Init
    for i  in  range(0, len(phase)):
        program[i], out_signal, pointer[i] = run_program(program[i], [phase[i], in_signal],
                                                         pause_on_out=True, verbose=False)
        in_signal = out_signal[0]

    #Feed back
    while True:
        for i in range(0, len(phase)):
            program[i], out_signal, pointer[i] = run_program(program[i], in_signal, i=pointer[i],
                                                             pause_on_out=True, verbose=False)
            if pointer[i] == -1:
                break
            in_signal = out_signal[0]
        if pointer[i] == -1:
            break
            
    if in_signal > max_out:
        max_out = in_signal
        
print('Part 2 Solution:', max_out)

Part 2 Solution: 8754464
